# # Chord Progressions Notebook
En este notebook:

1. Generamos archivos .mid con `generate_progression.py` en TODAS las tonalidades (p. ej. "A-1-..." o "C#-5-...").
2. Convertimos esos .mid a .wav con Timidity usando `audio_conversion.py`.
3. Creamos .jams parseando el nombre de cada .mid para extraer:
   - Nota y octava (p. ej. "A-1"),
   - Progresión en numerales romanos (p. ej. "I-vi-IV-V"),
   - Índice final, p. ej. "3".

---

## 1. Imports y Configuración

In [1]:
import sys
from pathlib import Path

# Ajustar la ruta para que Python reconozca 'src'
PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.config import MIDI_DIR, WAV_DIR, JAMS_DIR, DEFAULT_TEMPO
from src.generate_progression import generate_progression
from src.audio_conversion import convert_all_mid_in_folder
from src.jams_creation import create_jams_file

print("MIDI_DIR =", MIDI_DIR)
print("WAV_DIR  =", WAV_DIR)
print("JAMS_DIR =", JAMS_DIR)
print("DEFAULT_TEMPO =", DEFAULT_TEMPO)


MIDI_DIR = /home/cepatinog/MIR/final_project/chord_progression_generator/data/midi
WAV_DIR  = /home/cepatinog/MIR/final_project/chord_progression_generator/data/wav
JAMS_DIR = /home/cepatinog/MIR/final_project/chord_progression_generator/data/jams
DEFAULT_TEMPO = 60


## 2. Generar Progresiones .mid
Probaremos un par de progresiones:
- `ii,7-V,7-I,maj7` (3 acordes)
- `I-vi-IV-V` (4 acordes)

Cada una se expandirá en las diferentes octavas y tonalidades, generando múltiples .mid en `data/midi/...`.


In [ ]:
# Ejemplo 1
prog_1 = "ii,7-V,7-I,maj7"
name_1 = "ii7-V7-Imaj7_strip"

generate_progression(
    progression=prog_1,
    name=name_1,
    output_dir=MIDI_DIR,
    tempo=DEFAULT_TEMPO
)

# Ejemplo 2
prog_2 = "I-vi-IV-V"
name_2 = "I-vi-IV-V_strip"

generate_progression(
    progression=prog_2,
    name=name_2,
    output_dir=MIDI_DIR,
    tempo=DEFAULT_TEMPO
)

print("Progresiones .mid generadas en data/midi/")



In [ ]:
midi_folder_1 = MIDI_DIR / name_1
midi_folder_2 = MIDI_DIR / name_2

## 3. Crear .jams parseando el nombre de cada .mid y limpiando subguiones
No asumimos una sola tonalidad. En su lugar:
- extraemos `<noteName>-<oct>` como tonalidad (ej: `Eb-6` => `Eb`),
- extraemos la progresión entre `parts[2:-1]`,
- dividimos los tokens y cada uno lo limpiamos con `strip_suffix` para quitar `_algo`.



In [ ]:
def strip_suffix(token: str) -> str:
    """
    Elimina cualquier subguion '_' en el token.
    'Imaj7_test' -> 'Imaj7'
    'V,7_ex'     -> 'V,7'
    """
    if "_" in token:
        return token.split("_", 1)[0]
    return token

def parse_filename_and_create_jams(midi_file: Path):
    """
    Recibe un archivo .mid, parsea su nombre,
    determina la tonalidad y secuencia de acordes,
    llama a create_jams_file con los tokens limpios.
    """
    base_name = midi_file.stem  # e.g. "Eb-6-ii7-V7-Imaj7_test-63"
    parts = base_name.split("-")
    if len(parts) < 3:
        print(f"Nombre de archivo inesperado => {base_name}")
        return

    # 1) Nota + octava => p.ej. "Eb-6"
    note_plus_oct = "-".join(parts[:2])  # ["Eb","6"] => "Eb-6"
    # 2) Progresión => parts[2:-1], sin el índice final
    progression_list = parts[2:-1]       # e.g. ["ii7","V7","Imaj7_test"]
    # 3) index => parts[-1], no se usa directamente
    idx_str = parts[-1]

    # Reconstruir la progresión => "ii7-V7-Imaj7_test"
    progression_str = "-".join(progression_list)
    raw_tokens = progression_str.split("-")  # ["ii7","V7","Imaj7_test"]
    # Aplicamos strip_suffix
    roman_sequence = [strip_suffix(tok) for tok in raw_tokens] 
    # => ["ii7","V7","Imaj7"]

    # Parse la tonalidad => e.g. "Eb-6" => ["Eb","6"] => "Eb"
    splitted = note_plus_oct.split("-")
    if len(splitted) == 2:
        tonalidad = splitted[0]  # "Eb"
    else:
        tonalidad = splitted[0]

    # Llamamos create_jams_file
    jam_name = base_name
    jam_path = create_jams_file(
        roman_sequence=roman_sequence,
        key=tonalidad,
        jam_name=jam_name,
        progression_name="-".join(roman_sequence)
    )
    print(f"[JAMS] {midi_file.name} => {jam_path.name}")


def parse_and_create_jams_in_folder(folder: Path):
    """
    Aplica parse_filename_and_create_jams a cada .mid de la carpeta.
    """
    if not folder.exists():
        print(f"No existe carpeta => {folder}")
        return
    mid_files = list(folder.rglob("*.mid"))
    if not mid_files:
        print(f"No hay .mid en => {folder}")
        return

    for mf in mid_files:
        parse_filename_and_create_jams(mf)

# Procesamos ambas carpetas
parse_and_create_jams_in_folder(midi_folder_1)
parse_and_create_jams_in_folder(midi_folder_2)


## 4. Verificar .jams



In [ ]:
for jfile in (JAMS_DIR / name_1).rglob("*.jams"):
    print("[JAMS #1]", jfile)

for jfile in (JAMS_DIR / name_2).rglob("*.jams"):
    print("[JAMS #2]", jfile)



## 5. Convertir a WAV con Timidity


In [ ]:

convert_all_mid_in_folder(midi_folder_1)
convert_all_mid_in_folder(midi_folder_2)

print("Archivos .wav creados en data/wav/")